In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install torchao==0.14.0 executorch pytorch_tokenizers

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import concatenate_datasets
import numpy as np

fourbit_models = [
    "unsloth/Qwen3-4B",

]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = 1024,
    full_finetuning = True,
    qat_scheme = "phone-deployment",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: You selected full finetuning support, but 4bit / 8bit is enabled - disabling LoRA / QLoRA.
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Applying QAT to mitigate quantization degradation


In [ ]:
from datasets import load_dataset

raw = load_dataset("igorktech/anekdots", split="train")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/497596 [00:00<?, ? examples/s]

### Чистка
Исходный датасет содержит очень разные анекдоты, его нужно немного почистить    
1) Берем 70 процентов анекдотов усредненной длины, 15 длинных, 15 коротких
2) Чистим от мусрора    
3) Проверяем, чтобы анекдоты были на русском  
4) Забиваем итоговую выборку  

In [ ]:
raw = raw.select(range(50000))

_word = re.compile(r"[A-Za-zА-Яа-яЁё0-9]+(?:-[A-Za-zА-Яа-яЁё0-9]+)*", re.UNICODE)

def word_len(text: str) -> int:
    return len(_word.findall(text or ""))

lengths = [word_len(x["text"]) for x in raw]
p25 = int(np.percentile(lengths, 25))
p75 = int(np.percentile(lengths, 75))

_lat = re.compile(r"[A-Za-z]")
def is_ok(text: str) -> bool:
    if not text:
        return False
    t = " ".join(text.split())
    if len(t) < 40:
        return False
    if "http" in t.lower() or "www" in t.lower():
        return False
    if _lat.search(t):
        return False
    if word_len(t) < 10:
        return False
    return True

raw = raw.filter(lambda x: is_ok(x["text"]))

def bucket(example):
    n = word_len(example["text"])
    if n <= p25:
        return "short"
    elif n >= p75:
        return "long"
    else:
        return "medium"

raw_b = raw.map(lambda x: {"bucket": bucket(x)})

short_ds  = raw_b.filter(lambda x: x["bucket"] == "short")
medium_ds = raw_b.filter(lambda x: x["bucket"] == "medium")
long_ds   = raw_b.filter(lambda x: x["bucket"] == "long")


N = min(50000, len(raw_b))
n_medium = int(N * 0.70)
n_short  = int(N * 0.15)
n_long   = N - n_medium - n_short

short_sel  = short_ds.shuffle(seed=42).select(range(min(n_short,  len(short_ds))))
medium_sel = medium_ds.shuffle(seed=42).select(range(min(n_medium, len(medium_ds))))
long_sel   = long_ds.shuffle(seed=42).select(range(min(n_long,   len(long_ds))))

raw = concatenate_datasets([short_sel, medium_sel, long_sel]).shuffle(seed=42)


In [ ]:
_ws = re.compile(r"\s+")
_word = re.compile(r"[A-Za-zА-Яа-яЁё0-9]+(?:-[A-Za-zА-Яа-яЁё0-9]+)*", re.UNICODE)

def split_by_first_n_words(joke: str, n_words: int = 5):
    if not joke:
        return None
    s = _ws.sub(" ", joke.strip())
    it = list(_word.finditer(s))
    if len(it) <= n_words:
        return None
    cut = it[n_words - 1].end()
    prefix = s[:cut].rstrip()
    suffix = s[cut:].lstrip()
    return prefix, suffix

def to_chat_text(examples):
    texts = []
    for joke in examples["text"]:
        out = split_by_first_n_words(joke, n_words=5)
        if out is None:
            continue
        prefix, suffix = out
        msgs = [
            {"role": "system", "content": "Ты дописываешь анекдоты на русском. Не повторяй затравку — продолжай с того места."},
            {"role": "user", "content": f"Продолжи анекдот. Начало:\n{prefix}"},
            {"role": "assistant", "content": suffix},
        ]
        texts.append(tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False))
    return {"text": texts}

train_ds = raw.map(
    to_chat_text,
    batched=True,
    remove_columns=raw.column_names,
)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer, SFTConfig



trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    #formatting_func=formatting_func,
    train_dataset=train_ds,
    eval_dataset=None,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4, # размер батча 2*4, градиент суммируется 4 шага подряд перед обновлением
        warmup_steps=5,
        max_steps=70,
        learning_rate=5e-5,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.001, # регуляризация
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)

trainer_stats = trainer.train()


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/48902 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 48,902 | Num Epochs = 1 | Total steps = 70
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 751,632,384 of 751,632,384 (100.00% trained)


Step,Training Loss
10,2.097300
20,1.889800
30,2.132700
40,1.920600
50,1.937900
60,1.937500
70,1.821300


In [ ]:
import torch

model.eval()

PREFIXES_PATH = "/content/prefixes.txt"
OUTPUT_PATH = "/content/output.txt"

THINK_RE = re.compile(
    r"(assistant\s*)?<think>.*?</think>",
    flags=re.DOTALL | re.IGNORECASE,
)

def clean_text(text: str) -> str:
    text = THINK_RE.sub("", text)

    text = re.sub(r"^\s*assistant\s*", "", text, flags=re.IGNORECASE)

    text = re.sub(r"\s+", " ", text).strip()
    return text


def generate_body(prefix: str) -> str:
    prompt = f"Продолжи анекдот по-русски:\n{prefix}"

    messages = [{"role": "user", "content": prompt}]
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.9,
            top_p=0.95,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=False,
        )

    decoded = tokenizer.decode(out[0], skip_special_tokens=True)

    body = decoded.split(prefix, 1)[-1].strip()

    body = re.sub(r"\s+", " ", body).strip()
    body = clean_text(body)
    return body


with open(PREFIXES_PATH, "r", encoding="utf-8") as f:
    prefixes = [line.strip() for line in f if line.strip()]

with open(OUTPUT_PATH, "w", encoding="utf-8") as out_f:
    for idx, prefix in enumerate(prefixes):
        body = generate_body(prefix)
        out_f.write(f"{prefix} {body}\n")
        print(f" {prefix} {body}\n")

print(f"Сохранено в {OUTPUT_PATH}")


 1 Идёт мужик по лесу . Ставит на куст, походит баба. - А ты кто? - Вчера вчера.

 2 Встречаются два друга . Один: - Доктор, а вам вчера с кем не ходили? Второй: - Да вчера идет биологический магазин. А вот я вчера пил...

 3 Приходят мужик в бар . - Вчера познакомились. Нет, не вечеру и утро. - Муж, вчера ночью, я пойду домой на магазин. Слушай, как вы там заходите? - Поручит мне

 4 Жена говорит мужу . - Надо срочно пересечу руки и ноги... Муж женатый: - Слушай, не усмейся! Я тебя захотел пересечать...

 5 Приходят альфа, бета и гамма в бар . Один садиться - другой раз - третий раз...

 6 Идёт медведь по лесу , и видит впередний пустын. Грустный медведь подошепеняет: - Папа, я не понял... Прописал!

 7 Приходит мужик к врачу . - Ты не знаешь, что я мужиков не люблю? - Давай попробую. - Как бы... - А вы поймали...

 8 Встречаются русский, американец и немец : - Как бы в России не было жениха? - Сын, что я вытаскал?!

 9 Идёт по улице девушка на салоне и встречает водку, ебет себе. Дев

In [ ]:
import gc, torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
